<a href="https://colab.research.google.com/github/AntonYermilov/deep-unsupervised-learning/blob/task2/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn import NLLLoss, CrossEntropyLoss
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange

import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px

In [0]:
def generate_dataset(size=100000):
    p = np.load('distribution.npy')
    n, m = p.shape
    p = p.flatten()
    data = np.random.choice(np.arange(n * m), p=p, size=size).astype(np.int).reshape(-1, 1)
    data = np.hstack((data // n, data % n))
    return data

In [0]:
data = generate_dataset()

In [0]:
train_input, val_input = train_test_split(data, random_state=1, test_size=0.2)
train_input, val_input = torch.tensor(train_input), torch.tensor(val_input)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

## MLP

In [0]:
class SoftmaxModel:
    def __init__(self, n, m, epochs, batch_size):
        self.n = n
        self.m = m
        self.theta = torch.randn(n)
        self.theta.requires_grad = True
        self.batch_size = batch_size
        self.epochs = epochs

        self.optimizer = Adam([self.theta], lr=0.001, betas=(0.9, 0.999), eps=1e-8)
        self.loss_fn = NLLLoss()

        # self.best_params = (self.theta.parameters()
        self.train_losses = []
        self.val_losses = []
    
    def _fit(self, train_dataloader, val_dataloader):
        print('=== Fitting Softmax Model ===')

        for epoch in range(self.epochs):
            nlll_train, nlll_val = 0, 0

            for i, batch in enumerate(train_dataloader):
                target = batch[0][:,0]
                self.optimizer.zero_grad()

                pred = F.log_softmax(self.theta)
                pred = pred.repeat(target.shape[0], 1)
                loss = self.loss_fn(pred, target)
                nlll_train += loss.item()

                loss.backward()
                self.optimizer.step()

            for i, batch in enumerate(val_dataloader):
                target = batch[0][:,0]

                with torch.no_grad():
                    pred = F.log_softmax(self.theta)
                    pred = pred.repeat(target.shape[0], 1)
                    loss = self.loss_fn(pred, target)

                nlll_val += loss.item()

            self.train_losses.append(nlll_train / len(train_dataloader))
            self.val_losses.append(nlll_val / len(val_dataloader))
            if (epoch + 1) % 1 == 0:
                print(f'Epoch {epoch + 1}: NLLL_Train={self.train_losses[-1]:.8f}, NLLL_Val={self.val_losses[-1]:.8f}')

        return self.train_losses, self.val_losses

    def fit(self, train_input, val_input):
        train_data = TensorDataset(train_input)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.batch_size)

        val_data = TensorDataset(val_input)
        val_sampler = SequentialSampler(val_data)
        val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=val_input.shape[0])
        
        self._fit(train_dataloader, val_dataloader)


class ConditionedMLPModel:
    def __init__(self, n, m, epochs, batch_size):
        self.n = n
        self.m = m
        self.theta = nn.Sequential(
            nn.Linear(n, 100),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(100, m),
            nn.Sigmoid(),
            nn.LogSoftmax()
        )
        self.theta.cuda()
        self.batch_size = batch_size
        self.epochs = epochs
        self.one_hot = torch.eye(self.n)

        self.optimizer = Adam(self.theta.parameters(), lr=0.0005, betas=(0.9, 0.999), eps=1e-8)
        self.loss_fn = NLLLoss()

        # self.best_params = self.theta.parameters()
        self.train_losses = []
        self.val_losses = []

    def _fit(self, train_dataloader, val_dataloader):
        print('=== Fitting MLP Model ===')

        for epoch in range(self.epochs):
            nlll_train, nlll_val = 0, 0

            self.theta.train()

            for i, batch in enumerate(train_dataloader):
                input = self.one_hot[batch[0][:,0]].to(device)
                target = batch[0][:,1].to(device)
                self.optimizer.zero_grad()

                pred = self.theta(input)
                loss = self.loss_fn(pred, target)
                nlll_train += loss.item()

                loss.backward()
                self.optimizer.step()

            self.theta.eval()

            for i, batch in enumerate(val_dataloader):
                input = self.one_hot[batch[0][:,0]].to(device)
                target = batch[0][:,1].to(device)

                with torch.no_grad():
                    pred = self.theta(input)
                    loss = self.loss_fn(pred, target)

                nlll_val += loss.item()

            self.train_losses.append(nlll_train / len(train_dataloader))
            self.val_losses.append(nlll_val / len(val_dataloader))
            if (epoch + 1) % 1 == 0:
                print(f'Epoch {epoch + 1}: NLLL_Train={self.train_losses[-1]:.8f}, NLLL_Val={self.val_losses[-1]:.8f}')

        return self.train_losses, self.val_losses

    def fit(self, train_input, val_input):
        train_data = TensorDataset(train_input)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.batch_size)

        val_data = TensorDataset(val_input)
        val_sampler = SequentialSampler(val_data)
        val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=val_input.shape[0])
        
        self._fit(train_dataloader, val_dataloader)

In [77]:
model1 = SoftmaxModel(200, 200, 15, 32)
model2 = ConditionedMLPModel(200, 200, 45, 1024)

model1.fit(train_input, val_input)
model2.fit(train_input, val_input)

=== Fitting Softmax Model ===


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Epoch 1: NLLL_Train=5.54417234, NLLL_Val=5.38948107
Epoch 2: NLLL_Train=5.33543625, NLLL_Val=5.30587769
Epoch 3: NLLL_Train=5.29220488, NLLL_Val=5.28886366
Epoch 4: NLLL_Train=5.28357975, NLLL_Val=5.28589725
Epoch 5: NLLL_Train=5.28212879, NLLL_Val=5.28533220
Epoch 6: NLLL_Train=5.28193328, NLLL_Val=5.28534126
Epoch 7: NLLL_Train=5.28186008, NLLL_Val=5.28547287
Epoch 8: NLLL_Train=5.28188181, NLLL_Val=5.28516483
Epoch 9: NLLL_Train=5.28187606, NLLL_Val=5.28532267
Epoch 10: NLLL_Train=5.28187408, NLLL_Val=5.28550529
Epoch 11: NLLL_Train=5.28189317, NLLL_Val=5.28532553
Epoch 12: NLLL_Train=5.28187675, NLLL_Val=5.28532457
Epoch 13: NLLL_Train=5.28186787, NLLL_Val=5.28540707
Epoch 14: NLLL_Train=5.28188210, NLLL_Val=5.28530025
Epoch 15: NLLL_Train=5.28187670, NLLL_Val=5.28538275
=== Fitting MLP Model ===


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Epoch 1: NLLL_Train=5.29650441, NLLL_Val=5.29486275
Epoch 2: NLLL_Train=5.29018775, NLLL_Val=5.28691626
Epoch 3: NLLL_Train=5.27849200, NLLL_Val=5.27747011
Epoch 4: NLLL_Train=5.26977255, NLLL_Val=5.27415466
Epoch 5: NLLL_Train=5.26646350, NLLL_Val=5.27338076
Epoch 6: NLLL_Train=5.26540677, NLLL_Val=5.27297735
Epoch 7: NLLL_Train=5.26390145, NLLL_Val=5.27243376
Epoch 8: NLLL_Train=5.26279968, NLLL_Val=5.27198458
Epoch 9: NLLL_Train=5.26215096, NLLL_Val=5.27152538
Epoch 10: NLLL_Train=5.26095657, NLLL_Val=5.27106142
Epoch 11: NLLL_Train=5.26019637, NLLL_Val=5.27074146
Epoch 12: NLLL_Train=5.25906884, NLLL_Val=5.27030325
Epoch 13: NLLL_Train=5.25826009, NLLL_Val=5.26999521
Epoch 14: NLLL_Train=5.25733420, NLLL_Val=5.26967716
Epoch 15: NLLL_Train=5.25696288, NLLL_Val=5.26933718
Epoch 16: NLLL_Train=5.25580064, NLLL_Val=5.26901293
Epoch 17: NLLL_Train=5.25405164, NLLL_Val=5.26866627
Epoch 18: NLLL_Train=5.25366894, NLLL_Val=5.26835918
Epoch 19: NLLL_Train=5.25224217, NLLL_Val=5.26797533
Ep

In [0]:
def get_samples(model1, model2, size):
    p1 = F.softmax(model1.theta).detach().numpy()
    x1 = np.random.choice(np.arange(p1.shape[0]), p=p1, size=size)
    E = torch.eye(p1.shape[0])
    x2 = model2.theta(E[x1].to(device)).cpu().detach().numpy()
    x2 = np.array([np.random.choice(np.arange(x2.shape[1]), p=np.exp(x2[i])) for i in range(x2.shape[0])])
    assert x1.shape[0] == size
    assert x2.shape[0] == size
    return x1, x2

In [79]:
samples = get_samples(model1, model2, 100000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



In [0]:
def plot_hist(title, x, y):
    figure = go.Figure(data=[
        go.Histogram2d(x=y, y=199-x, xbins=dict(start=0, end=200, size=1), ybins=dict(start=0, end=200, size=1))
    ])
    # figure.update_layout(title='Distribution', barmode='stack')
    figure.update_layout(title=title, height=600, width=600)
    figure.show()

In [81]:
plot_hist('Input Samples', data[:,0], data[:,1])

In [82]:
plot_hist('Generated Samples', samples[0], samples[1])

## MADE